<h> Do NOT run the following blocks </h>

In [79]:
file_path = r"C:/Users/huda/Downloads/part_1.txt"
benchmark_base_path = "C:/Users/huda/Documents/GitHub/scheduling_model/code/external_test_data/FJSSPinstances/"
file = open(file_path, 'r')
file_content = file.readlines()

In [80]:
results_30min = file_content[:402]

In [91]:
#NOTE: ONLY NEEDED TO BE DONE ONCE TO DIVIDE DATA INTO THE FILES
file_path = r"C:\Users\huda\Downloads\part_2.txt"
file = open(file_path, 'r')
file_content = file.readlines()
results_60min = file_content
target_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\gurobi_results\runs_60min\results.txt'
file = open(target_path, 'a')
for line in results_60min:
    file.write(line)
file.close()

In [89]:
#NOTE: ONLY NEEDED TO BE DONE ONCE TO DIVIDE DATA INTO THE FILES
# rewrite data

target_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\gurobi_results\runs_30min\results.txt'
file = open(target_path, 'w')
for line in results_30min:
    # TODO: remove x, y, c vectors
    file.write(line)
file.close()
target_path = r'C:\Users\huda\Documents\GitHub\scheduling_model\code\reworked_data_model\results\gurobi_results\runs_60min\results.txt'
file = open(target_path, 'w')
for line in file_content[402:]:
    file.write(line)
file.close()

<h> Run from here</h>

In [92]:
import matplotlib.pyplot as plt

In [93]:
file_path = r"C:/Users/huda/Downloads/part_1.txt"
benchmark_base_path = "C:/Users/huda/Documents/GitHub/scheduling_model/code/external_test_data/FJSSPinstances/"
file = open(file_path, 'r')
file_content = file.readlines()

In [94]:
results_30min = file_content[:402]

In [95]:
def read_benchmark_file(source, instance):
    if source.startswith('0'):
        target_file = f'Behnke{instance}.fjs'
    elif source.startswith('1'):
        target_file = f'BrandimarteMk{instance}.fjs'
    elif source.startswith('2a'):
        target_file = f'HurinkSdata{instance}.fjs'
    elif source.startswith('2b'):
        target_file = f'HurinkEdata{instance}.fjs'
    elif source.startswith('2c'):
        target_file = f'HurinkRdata{instance}.fjs'
    elif source.startswith('2d'):
        target_file = f'HurinkVdata{instance}.fjs'
    elif source.startswith('3'):
        target_file = f'DPpaulli{instance}.fjs'
    elif source.startswith('4'):
        target_file = f'ChambersBarnes{instance}.fjs'
    elif source.startswith('5'):
        target_file = f'Kacem{instance}.fjs'
    elif source.startswith('6'):
        target_file = f'Fattahi{instance}.fjs'
    path = benchmark_base_path + f'{source}\\{target_file}'
    return open(path, 'r').readlines()

In [96]:
results = []
success_count = 0
timeout_count = 0
other_limit_count = 0
successful = []
failed = []
for row in results_30min:
    line = dict()
    columns = row.split(';')
    line["source"] = columns[0]
    line["instance"] = int(columns[1])
    line["best_objective"] = float(columns[2])
    line["upper_bound"] = float(columns[3])
    line["lower_bound"] = float(columns[4])
    line["gap"] = float(columns[5])
    line["status"] = int(columns[6])
    line["n_explored"] = int(float(columns[7]))
    line["runtime"] = float(columns[8])
    line["overall_runtime"] = float(columns[9])
    line["setup_time"] = line["overall_runtime"] - line["runtime"]
    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line["source"], line["instance"])
    benchmark_system = benchmark_file[0].split(' ')
    line["n_machines"] = int(benchmark_system[1])
    line["n_jobs"] = int(benchmark_system[0])
    line["average_workstations_per_operation"] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line["n_operations"] = sum([int(x.split(' ')[0]) for x in jobs])
    line["average_operations_per_job"] = line["n_operations"] / line["n_jobs"]
    line["beta_flexibility"] = line["average_workstations_per_operation"]/line["n_machines"]
    line["approximate_max_dissimilarity"] = line["n_operations"] + line["average_operations_per_job"] * line["average_workstations_per_operation"]
    
    results.append(line)
successful = [x for x in results if x["status"] == 2]
failed = [x for x in results if x["status"] != 2]

In [97]:
def average(values):
    return sum(values) / len(values)

In [98]:
n_machines_success = [x["n_machines"] for x in successful]
n_machines_failed = [x["n_machines"] for x in failed]
n_jobs_success = [x["n_jobs"] for x in successful]
n_jobs_failed = [x["n_jobs"] for x in failed]
n_operations_success = [x["n_operations"] for x in successful]
n_operations_failed = [x["n_operations"] for x in failed]
success_beta = [x["beta_flexibility"] for x in successful]
failed_beta = [x["beta_flexibility"] for x in failed]
success_approximated_max_dissimilarity = [x["approximate_max_dissimilarity"] for x in successful]
failed_approximated_max_dissimilarity = [x["approximate_max_dissimilarity"] for x in failed]
print(f'Averages of\tSuccess(n={len(successful)})\t\tFailed(n={len(failed)})')
print(f'# machines:\t{average(n_machines_success)}\t{average(n_machines_failed)}')
print(f'# jobs:\t\t{average(n_jobs_success)}\t{average(n_jobs_failed)}')
print(f'# operations:\t{average(n_operations_success)}\t{average(n_operations_failed)}')
print(f'Flexibility:\t{average(success_beta)}\t{average(failed_beta)}')
print(f'Approx.Max.Dis.:{average(success_approximated_max_dissimilarity)}\t{average(failed_approximated_max_dissimilarity)}')


Averages of	Success(n=166)		Failed(n=236)
# machines:	10.75301204819277	15.453389830508474
# jobs:		9.885542168674698	25.23728813559322
# operations:	89.18072289156626	192.97457627118644
Flexibility:	0.27365994654805786	0.27735930966704997
Approx.Max.Dis.:111.164969879518	220.99678672316375


In [107]:
gap_failed = [x["gap"] for x in failed]
average_gap_failed = average(gap_failed)
print(f"Average Gap in failed runs: {average_gap_failed}")
benchmarks_by_gap = sorted(failed, key=lambda x: x["gap"])
print(f'Max Gap: {benchmarks_by_gap[-1]["gap"]} for instance {benchmarks_by_gap[-1]["source"]}-{benchmarks_by_gap[-1]["instance"]}')
print(f'Min Gap: {benchmarks_by_gap[0]["gap"]} for instance {benchmarks_by_gap[0]["source"]}-{benchmarks_by_gap[0]["instance"]}')



Average Gap in failed runs: 0.39115123202534696
Max Gap: 0.8843441466854631 for instance 0_BehnkeGeiger-60


Min Gap: 0.011494244358202708 for instance 0_BehnkeGeiger-42


In [112]:
success_divided = dict()
failed_divided = dict()
sources = []
for instance in successful:
    if instance["source"] not in success_divided:
        success_divided[instance["source"]] = [x for x in successful if x["source"] == instance["source"]]
        if instance["source"] not in sources:
            sources.append(instance["source"])
for instance in failed:
    if instance["source"] not in failed_divided:
        failed_divided[instance["source"]] = [x for x in failed if x["source"] == instance["source"]]
        if instance["source"] not in sources:
            sources.append(instance["source"])

In [113]:
for source in sources:
    if source not in success_divided:
        print(f'All instances of {source} failed.')
    if source not in failed_divided:
        print(f'All instances of {source} succeeded.')

All instances of 3_DPpaulli failed.


In [115]:
n_success_failed = dict()
for source in sources:
    if source not in n_success_failed:
        n_success_failed[source] = [0, 0]
    if source in success_divided:
        n_success_failed[source][0] = len(success_divided[source])
    if source in failed_divided:
        n_success_failed[source][1] = len(failed_divided[source])

end


In [127]:
print(f'source\t\t\tsuccess\tfailed\tsuccess/failure\t\t% success\t\t% failed')
for source in sources:
    print(f'{source}(n={sum(n_success_failed[source])})\t{n_success_failed[source][0]}\t{n_success_failed[source][1]}\t{n_success_failed[source][0] / n_success_failed[source][1]}\t{n_success_failed[source][0]/sum(n_success_failed[source])}\t{n_success_failed[source][1]/sum(n_success_failed[source])}')


source			success	failed	success/failure		% success		% failed
0_BehnkeGeiger(n=60)	9	51	0.17647058823529413	0.15	0.85
1_Brandimarte(n=15)	2	13	0.15384615384615385	0.13333333333333333	0.8666666666666667
2a_Hurink_sdata(n=66)	32	34	0.9411764705882353	0.48484848484848486	0.5151515151515151
2b_Hurink_edata(n=66)	35	31	1.1290322580645162	0.5303030303030303	0.4696969696969697
2c_Hurink_rdata(n=66)	24	42	0.5714285714285714	0.36363636363636365	0.6363636363636364
2d_Hurink_vdata(n=66)	27	39	0.6923076923076923	0.4090909090909091	0.5909090909090909
4_ChambersBarnes(n=21)	16	5	3.2	0.7619047619047619	0.23809523809523808
5_Kacem(n=4)	3	1	3.0	0.75	0.25
6_Fattahi(n=20)	18	2	9.0	0.9	0.1
3_DPpaulli(n=18)	0	18	0.0	0.0	1.0
